In [1]:
import import_ipynb
# import title_genre_predict
import preprocess_utils
import parse_utils
import wiki_kw_retrieve
import wikt_def_gen

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
import pyfiglet
import os
import numpy as np
import pickle

importing Jupyter notebook from preprocess_utils.ipynb
importing Jupyter notebook from parse_utils.ipynb


[nltk_data] Downloading package punkt to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term


[nltk_data] Downloading package benepar_en2 to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package benepar_en2 is already up-to-date!


importing Jupyter notebook from wiki_kw_retrieve.ipynb
importing Jupyter notebook from wikt_def_gen.ipynb


[nltk_data] Downloading package punkt to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [2]:
# print(pyfiglet.figlet_format("DAIC GLOGEN"),)
# print('DESCRIPTION: GLOGEN automatically generates glossaries and prepends them to given .txt files. \nENSURE: <filename>.txt == original text title.')
# answer = ''
# txt_address = ''
# while True:
#     answer = input('\nType "yes" / "no" to starting GLOGEN:\n>>>')
#     if answer.lower().startswith("y"):
#         txt_address += input('Type address where .txt files are stored. If same address as main.py press ENTER.\n>>>')
#         break
#     elif answer.lower().startswith("n"):
#         exit()
#     else:
#         print('INVALID INPUT --- TRY AGAIN.')
#         continue
# if txt_address == '':
#     txt_address = os.getcwd()

In [3]:
user_input_data = []

txt_address = '/home/jackragless/projects/github/DAIC_GLOGEN/text_files'

    
for filename in os.listdir(txt_address):
    if filename.endswith('.txt'):
        orig_text = open(txt_address+'/'+filename).read()
        
        processed = preprocess_utils.clean_text(orig_text)
        
        candidate_phrases = []
        pos = []
        for sent in nltk.sent_tokenize(processed):
            temp_tree = parse_utils.parseSent(sent)
            candidate_phrases.append( parse_utils.getPhraseNodes(temp_tree,[]) )
            pos.append( parse_utils.getWordNodes(temp_tree,[]) )
        
        user_input_data.append({
             'title':filename[:-4], 
#              'genre_preds':title_genre_predict.inference(filename[:-4]), 
             'keywords':np.nan, 
             'glossary':np.nan, 
             'raw_text':orig_text, 
             'pos':pos,
             'candidate_phrases':candidate_phrases
            })

In [4]:
user_input_data

[{'title': 'Australia',
  'keywords': nan,
  'glossary': nan,
  'raw_text': 'Australia, officially the Commonwealth of Australia, is a sovereign country comprising the mainland of the Australian continent, the island of Tasmania, and numerous smaller islands. It is the largest country in Oceania and the world\\\'s sixth-largest country by total area. The population of 26 million is highly urbanised and heavily concentrated on the eastern seaboard. Australia\\\'s capital is Canberra, and its largest city is Sydney. The country\\\'s other major metropolitan areas are Melbourne, Brisbane, Perth, and Adelaide.\\nIndigenous Australians inhabited the continent for about 65,000 years prior to the first arrival of Dutch explorers in the early 17th century, who named it New Holland. In 1770, Australia\\\'s eastern half was claimed by Great Britain and initially settled through penal transportation to the colony of New South Wales from 26 January 1788, a date which became Australia\\\'s national